In [ ]:
#@title Copyright 2024 Google LLC. { display-mode: "form" }
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License

In [ ]:
!pip install google-cloud-monitoring tabulate &> /dev/null

In [ ]:
import ee
import google
import grpc
import locale
import time

from google.colab import auth
from google.cloud import monitoring_v3 as monitoring_v3
from tabulate import tabulate

In [ ]:
auth.authenticate_user()
_ = locale.setlocale(locale.LC_ALL, '')

PROJECT_ID = "your-project-id" # @param {type:"string"}

credentials, _ = google.auth.default()
ee.Initialize(
    credentials,
    project=PROJECT_ID,
    opt_url='https://earthengine-highvolume.googleapis.com')

In [ ]:
class WorkloadResult:
  def __init__(self, workload_tag):
    self.workload_tag = workload_tag
    self.wall_seconds = 0
    self.interactive_eecu_seconds = 0
    self.batch_eecu_seconds = 0

  def getTotalEecuSeconds(self):
    """Return the total EECU-time footprint, in EECU-seconds."""
    return self.interactive_eecu_seconds + self.batch_eecu_seconds

  def getCost(self):
    """Return the estimated cost of the total EECU-time in USD."""
    INTERACTIVE_COST = 1.33 # USD / EECU-hour
    BATCH_COST = 0.40 # USD / EECU-hour
    return (
        self.interactive_eecu_seconds / 3600 * INTERACTIVE_COST + \
        self.batch_eecu_seconds / 3600 * BATCH_COST)

In [ ]:
def fetch_monitoring_workloads():
  # Configure a gRPC client for the Monitoring API.
  client = monitoring_v3.MetricServiceClient()
  project_name = f"projects/{PROJECT_ID}"

  # Establish the relevant time periods.
  now = time.time()
  seconds = int(now)
  nanos = int((now - seconds) * 10**9)
  lookback_hours = 24 # @param {type:"integer"}
  lookback_s = lookback_hours * 60 * 60
  alignment_period_s = 60 * 60

  interval = monitoring_v3.TimeInterval(
      {
          "end_time": {"seconds": seconds, "nanos": nanos},
          "start_time": {"seconds": (seconds - lookback_s), "nanos": nanos},
      }
  )

  aggregation = monitoring_v3.Aggregation(
      {
          "alignment_period": {"seconds": alignment_period_s},
          "per_series_aligner": monitoring_v3.Aggregation.Aligner.ALIGN_SUM,
          "group_by_fields": ["metric.labels.workload_tag", "metric.labels.compute_type"],
          "cross_series_reducer": monitoring_v3.Aggregation.Reducer.REDUCE_SUM
      }
  )

  results = client.list_time_series(
      request={
          "name": project_name,
          "filter": 'metric.type = "earthengine.googleapis.com/project/cpu/usage_time"',
          "interval": interval,
          "view": monitoring_v3.ListTimeSeriesRequest.TimeSeriesView.FULL,
          "aggregation": aggregation,
      }
  )

  # For each resulting timeseries, aggregate the data further by combining
  # different observations with the same `workload_tag` value.
  sums = {}
  for result in results:
    key = result.metric.labels['workload_tag'] or "[default]"
    if key not in sums:
      sums[key] = WorkloadResult(key)

    eecu_s = result.points[0].value.double_value

    # Different compute types have different costs. Track them separately, then
    # recompute the value
    match result.metric.labels['compute_type']:
      case 'batch':
        sums[key].batch_eecu_seconds += eecu_s
      case 'online' | 'highvolume':
        sums[key].interactive_eecu_seconds += eecu_s
      case _:
        raise RuntimeError(
            f"Unknown compute type: {result.metric.labels['compute_type']}")

  return sums

In [ ]:
# Gather the stats.
workload_stats = [[
      k.workload_tag,  # Workload tag
      k.getTotalEecuSeconds(),  # EECU-s
      locale.currency(k.getCost()) # Cost
    ] for k in fetch_monitoring_workloads().values()
  ]

# Print a nice table.
print(tabulate(
    workload_stats,
    headers=['Workload tag', 'EECU-seconds', 'Cost (USD)'],
    floatfmt=".2f"))

Workload tag      EECU-seconds  Cost (USD)
--------------  --------------  ------
foobar                   24.29  $0.01
[default]              1572.63  $0.58
